In [ ]:
import os

In [ ]:
os.chdir("/content/fsl")

In [ ]:
# import packages
from tqdm import tqdm

import torch as th
from torch import nn
from torch.optim import SGD, Adam
from torch.utils.data import DataLoader
from torch.nn.modules.loss import MSELoss

from torchvision.transforms import Compose, Grayscale, CenterCrop, ToTensor, ToPILImage, Resize
from torchvision.datasets import ImageFolder

from src import Learner, TaskSampler, RelationNetworks, CNNEncoder

In [ ]:
# configurations
CONFIG = {}
CONFIG['shot'] = 5
CONFIG['way'] = 5
CONFIG['query'] = 10
DEVICE = 'cuda' if th.cuda.is_available() else 'cpu'
if DEVICE is 'cuda':
  print ('cuda is available as device')

cuda is available as device


In [ ]:
# Image transformation pipeline

image_transform_pipe = Compose(
    [
        Grayscale(num_output_channels=3),
        CenterCrop(224),
        Resize(128),
        ToTensor()
    ]
)

# load Train dataset
train_dataset = ImageFolder(root = "./few_shot_data/train", transform=image_transform_pipe)
# load Validation dataset
val_dataset = ImageFolder(root = "./few_shot_data/val", transform=image_transform_pipe)
# load Test dataset
test_dataset = ImageFolder(root ="./few_shot_data/test", transform=image_transform_pipe)

In [ ]:
# Create task samplers
train_sampler = TaskSampler(train_dataset, n_way=CONFIG['way'], n_shot=CONFIG['shot'], n_query=CONFIG['query'], n_tasks=500)

validation_sampler = TaskSampler(val_dataset, n_way=CONFIG['way'], n_shot=CONFIG['shot'], n_query=CONFIG['query'], n_tasks=100)

test_sampler = TaskSampler(test_dataset, n_way=CONFIG['way'], n_shot=CONFIG['shot'], n_query=CONFIG['query'], n_tasks=20)

In [ ]:
# Create dataloaders
train_loader = DataLoader(
    train_dataset,
    batch_sampler=train_sampler,
    num_workers=8,
    pin_memory=True,
    collate_fn=train_sampler.episodic_collate_fn,
)

val_loader = DataLoader(
    val_dataset,
    batch_sampler=validation_sampler,
    num_workers=8,
    pin_memory=True,
    collate_fn=validation_sampler.episodic_collate_fn,
)

test_loader = DataLoader(
    test_dataset,
    batch_sampler=test_sampler,
    num_workers=8,
    pin_memory=True,
    collate_fn=test_sampler.episodic_collate_fn,
)

## Training first Relation Networks with SGD Optimizer

In [ ]:
# Load pretrained CNNEncoder, trained on miniImageNet on paper
pretrained_weights = th.load("./pretrained_model/miniimagenet_feature_encoder_5way_5shot.pkl", map_location=th.device('cpu'))
pretrained_encoder = CNNEncoder()
pretrained_encoder.load_state_dict(pretrained_weights)

model_1 = pretrained_encoder.cuda()
reln_classifier_1 = RelationNetworks(backbone=model_1, output_softmax_score=False)

In [ ]:
learning_rate = 0.05
optimizer_1 = SGD(classifier_1.parameters(), lr=learning_rate)
loss_function = MSELoss()
model_path = "./models/relnet/relnet100.pkl"
tensorboard_log_path = "./tensorboard/relnet/"

reln_c1_best_val_accuracy = Learner.fit(
    train_data_loader = train_loader,
    val_data_loader = val_loader,
    model = reln_classifier_1,
    optimizer=optimizer_1, 
    loss_function = loss_function,
    epochs = 100,
    model_path = model_path,
    tensorboard_log_path = tensorboard_log_path
  )

Training Epoch 0


100%|██████████| 500/500 [00:25<00:00, 19.94it/s, episode_accuracy=0.26, episode_loss=0.159, epoch_accuracy=0.243, epoch_loss=0.161]


Validating Epoch 0


100%|██████████| 100/100 [00:04<00:00, 22.17it/s, episode_accuracy=0.2, overall_accuracy=0.279]


Training Epoch 1


...

100%|██████████| 100/100 [00:05<00:00, 19.34it/s, episode_accuracy=0.66, overall_accuracy=0.7]


Training Epoch 99


100%|██████████| 500/500 [00:23<00:00, 21.40it/s, episode_accuracy=0.84, episode_loss=0.052, epoch_accuracy=0.794, epoch_loss=0.0733]


Validating Epoch 99


100%|██████████| 100/100 [00:04<00:00, 20.81it/s, episode_accuracy=0.68, overall_accuracy=0.714]


In [ ]:
print ("Relation Networks classifier 1 best validation accuracy: ", reln_c1_best_val_accuracy)

Relation Networks classifier 1 best validation accuracy:  0.7162


## Training second Relation Networks with Adam Optimizer

In [ ]:
# Load pretrained CNNEncoder, trained on miniImageNet on paper
pretrained_weights = th.load("./pretrained_model/miniimagenet_feature_encoder_5way_5shot.pkl", map_location=th.device('cpu'))
pretrained_encoder = CNNEncoder()
pretrained_encoder.load_state_dict(pretrained_weights)

model_2 = pretrained_encoder.cuda()
reln_classifier_2 = RelationNetworks(backbone=model_2, output_softmax_score=False)

In [ ]:
learning_rate = 0.05
optimizer_2 = Adam(classifier_2.parameters(), lr=learning_rate)
loss_function = MSELoss()
model_path = "./models/relnet/relnet100_2.pkl"
tensorboard_log_path = "./tensorboard/relnet/"

reln_c2_best_val_accuracy = Learner.fit(
    train_data_loader = train_loader,
    val_data_loader = val_loader,
    model = classifier_2,
    optimizer=optimizer_2, 
    loss_function = loss_function,
    epochs = 100,
    model_path = model_path,
    tensorboard_log_path = tensorboard_log_path
  )

Training Epoch 0


100%|██████████| 500/500 [00:23<00:00, 21.37it/s, episode_accuracy=0.58, episode_loss=0.129, epoch_accuracy=0.352, epoch_loss=0.152]


Validating Epoch 0


100%|██████████| 100/100 [00:04<00:00, 21.15it/s, episode_accuracy=0.64, overall_accuracy=0.476]


Training Epoch 1


...

100%|██████████| 100/100 [00:04<00:00, 21.32it/s, episode_accuracy=0.7, overall_accuracy=0.693]


Training Epoch 99


100%|██████████| 500/500 [00:23<00:00, 21.70it/s, episode_accuracy=0.84, episode_loss=0.0617, epoch_accuracy=0.745, epoch_loss=0.085]


Validating Epoch 99


100%|██████████| 100/100 [00:04<00:00, 20.09it/s, episode_accuracy=0.5, overall_accuracy=0.681]


In [ ]:
print ("Relation Networks classifier 2 best validation accuracy: ", reln_c2_best_val_accuracy)

Relation Networks classifier 2 best validation accuracy:  0.698


## Get test accuracy for each classifier

In [ ]:
# Get test scores of Relation Networks classifier 1

correct_prediction_count = 0
total_prediction_count = 0
for (support_images, support_labels, query_images, query_labels, _) in test_loader:
  prediction_labels = Learner.predict_images(
      support_images.to(DEVICE),
      support_labels.to(DEVICE),
      query_images.to(DEVICE),
      classifier_1
    )
  correct_prediction_count += (prediction_labels==query_labels).sum().item()
  total_prediction_count += len(prediction_labels)

overall_test_accuracy  = correct_prediction_count/total_prediction_count
print ("Relation Networks Classifier 1 test accuracy: ", overall_test_accuracy)

Relation Networks Classifier 1 test accuracy:  0.641


In [ ]:
# Get test scores of Relation Networks classifier 2
correct_prediction_count = 0
total_prediction_count = 0
for (support_images, support_labels, query_images, query_labels, _) in test_loader:
  prediction_labels = Learner.predict_images(
      support_images.to(DEVICE),
      support_labels.to(DEVICE),
      query_images.to(DEVICE),
      classifier_2
    )
  correct_prediction_count += (prediction_labels==query_labels).sum().item()
  total_prediction_count += len(prediction_labels)

overall_test_accuracy  = correct_prediction_count/total_prediction_count
print ("Relation Networks Classifier 2 test accuracy: ", overall_test_accuracy)

Relation Networks Classifier 2 test accuracy:  0.619
